Importación de Dataset 'IDH' para combinarlo con nuestro dataset original de Global Economy Indicators
=======================================

Importaremos los datos del IDH mediante WebScrapping

# Importamos librerías necesarias

In [ ]:
# Instalamos las librerías necesarias para el proyecto que encontraremos en el archivo requirements.txt
!pip install -q -r requirements.txt

# Para trabajar con datos
import pandas as pd

# Importamos la librería request para manejar las peticiones de acceso a la web y descarga de contenido.
import requests

# Importamos la librería BeautifulSoup para el manejo de los datos descargados.
from bs4 import BeautifulSoup

Obtenemos los datos del IDH en la wikipedia en inglés para obtener los datos de 2022 y tener los países en inglés

In [3]:
# Especificamos la web
enlace = r"https://en.wikipedia.org/wiki/List_of_countries_by_Human_Development_Index"

# Convertimos la web en texto
texto_web = requests.get(enlace).text

# Para darle un formato, BeautifulSoup tiene a su disposición un módulo para "formatear" o dar forma al contenido 
# usando el parser que presentamos anteriormente.
sopa = BeautifulSoup(texto_web, 'lxml')

# La información que queremos se encuentra en las tablas, vamos a recuperarlas
tablas = sopa.find_all('table')

# Al existir varias tablas, tenemos que localizarlas. La clase de CSS de las tablas es "wikitable sortable"
tablas_listados = sopa.find_all('table', class_= 'wikitable')

# Vamos a seleccionar la segunda tabla
IDH_PNUD_Principales = tablas_listados[1]

# Para localizar a los deportistas, vamos a transformar la tabla a una estructura dataframe de pandas
IDH_principal = pd.read_html(str(IDH_PNUD_Principales))[0]

# Guardamos la información en el archivo:
IDH_PNUD = pd.ExcelWriter('IDH_English_2022.xlsx', engine='xlsxwriter')
IDH_principal.to_excel(IDH_PNUD, sheet_name='2022')
IDH_PNUD.close() #el .save() dejó de funcionar, hay que usar .close()

Con este código, obtenemos una lista de países en inglés y su IDH en 2022, que se sale del rango de nuestros valores originales, por lo que vamos a buscar los datos de 2021 y los vamos a fusionar para que los países estén en inglés (como en nuestro dataframe) pero con datos de antes de 2022 (los últimos que haya disponibles en cada caso)

Repetimos el proceso con la wikipedia en español, que tiene más años

In [4]:
# Especificamos la web
enlace = r"https://es.wikipedia.org/wiki/Anexo:Países_por_índice_de_desarrollo_humano"

# Convertimos la web en texto
texto_web = requests.get(enlace).text

# Formateamos el contenido
sopa = BeautifulSoup(texto_web, 'lxml')

# Recuperamos las tablas
tablas = sopa.find_all('table')

# Localizamos las tablas de tipo "wikitable"
tablas_listados = sopa.find_all('table', class_= 'wikitable')

# Volvemos a seleccionar la segunda tabla
IDH_PNUD_Principales = tablas_listados[1]

# Transformamos la tabla a una estructura dataframe de pandas
IDH_principal = pd.read_html(str(IDH_PNUD_Principales))[0]

# Guardamos la información en el archivo:
IDH_PNUD = pd.ExcelWriter('IDH_Spanish_More.xlsx', engine='xlsxwriter')
IDH_principal.to_excel(IDH_PNUD, sheet_name='All')
IDH_PNUD.close() #el .save() dejó de funcionar, hay que usar .close()

Ahora tenemos que limpiar los archivos Excel para que no tengan columnas raras ni encabezados erróneos

In [5]:
English=pd.read_excel("C:\Users\Usuario\Documents\GitHub\Final-Project\Data\IDH_English_2022.xlsx")
Spanish=pd.read_excel("C:\Users\Usuario\Documents\GitHub\Final-Project\Data\IDH_Spanish_More.xlsx",header=1)

In [6]:
English.drop(columns=["Unnamed: 0", "Δ", "% annual growth (2010–2022)"], inplace=True)
English.drop(index=192, inplace=True)
English

,Rank,Country or territory,HDI value
0,1,Switzerland,0.967
1,2,Norway,0.966
2,3,Iceland,0.959
3,4,Hong Kong,0.956
4,5,Denmark,0.952
...,...,...,...
187,188,Mali,0.410
188,189,Niger,0.394
189,189,Chad,0.394
190,191,Central African Republic,0.387


In [7]:
Spanish.drop(columns=["Unnamed: 0", "Crecimiento (2021 - 2022)", "2015"],inplace=True)
Spanish.drop(index=[0,1,71,101,122,165,198],inplace=True)
Spanish["Rank"] = Spanish["#"].str.replace("°","").astype(int)
Spanish.drop(columns=["#"], inplace=True)
Spanish

,País,2022,2021,2020,2019,Rank
2,Suiza,0967,0965,0955,0962,1
3,Noruega,0966,0964,0959,0961,2
4,Islandia,0959,0957,0957,0960,3
5,Hong Kong,0956,0959,0949,0952,4
6,Dinamarca,0952,0948,0947,0946,5
...,...,...,...,...,...,...
193,Malí,0410,0408,0427,0433,188
194,Níger,0394,0387,0401,0406,189
195,Chad,0394,0393,0397,0403,189
196,República Centroafricana,0387,0387,0407,0411,191


In [8]:
IDH = pd.merge(English, Spanish, on='Rank', how='inner')

In [9]:
IDH.tail(20)

,Rank,Country or territory,HDI value,País,2022,2021,2020,2019
224,177,Liberia,0.487,Liberia,0487,0484,0480,0484
225,177,Liberia,0.487,Madagascar,0487,0484,0501,0510
226,177,Madagascar,0.487,Liberia,0487,0484,0480,0484
227,177,Madagascar,0.487,Madagascar,0487,0484,0501,0510
228,179,Guinea-Bissau,0.483,Guinea-Bisáu,0483,0482,0483,0490
229,180,DR Congo,0.481,República Democratica del Congo,0481,0479,0479,0482
230,181,Guinea,0.471,Guinea,0471,0467,0466,0467
231,182,Afghanistan,0.462,Afganistán,0462,0473,0483,0488
232,183,Mozambique,0.461,Mozambique,0461,0459,0453,0456
233,184,Sierra Leone,0.458,Sierra Leona,0458,0456,0475,0480


In [10]:
IDH.drop(index=[5,6,9,10,14,15,18,19,20,22,23,24,31,34,35,38,41,42,55,56,60,61,67,68,81,82,86,87,94,97,108,109,115,116,120,123,125,126,139,142,160,161,188,189,205,207,210,211,225,226,239,240], inplace=True)

In [11]:
IDH

,Rank,Country or territory,HDI value,País,2022,2021,2020,2019
0,1,Switzerland,0.967,Suiza,0967,0965,0955,0962
1,2,Norway,0.966,Noruega,0966,0964,0959,0961
2,3,Iceland,0.959,Islandia,0959,0957,0957,0960
3,4,Hong Kong,0.956,Hong Kong,0956,0959,0949,0952
4,5,Denmark,0.952,Dinamarca,0952,0948,0947,0946
...,...,...,...,...,...,...,...,...
237,188,Mali,0.410,Malí,0410,0408,0427,0433
238,189,Niger,0.394,Níger,0394,0387,0401,0406
241,189,Chad,0.394,Chad,0394,0393,0397,0403
242,191,Central African Republic,0.387,República Centroafricana,0387,0387,0407,0411


In [12]:
IDH_bien = IDH
IDH = IDH_bien.reset_index()
IDH.drop(columns=["Rank", "País", "HDI value","2022"], inplace=True)

In [13]:
IDH.drop(columns=["index"], inplace=True)

In [14]:
IDH

,Country or territory,2021,2020,2019
0,Switzerland,0965,0955,0962
1,Norway,0964,0959,0961
2,Iceland,0957,0957,0960
3,Hong Kong,0959,0949,0952
4,Denmark,0948,0947,0946
...,...,...,...,...
187,Mali,0408,0427,0433
188,Niger,0387,0401,0406
189,Chad,0393,0397,0403
190,Central African Republic,0387,0407,0411


In [15]:
IDH["2021"] = ((IDH["2021"]).astype(int))/1000
IDH["2020"] = ((IDH["2020"]).astype(int))/1000
IDH["2019"] = ((IDH["2019"]).astype(int))/1000

In [16]:
IDH.sort_values(by="Country or territory",inplace=True)
IDH.rename(columns={"Country or territory":"Country"}, inplace=True)
IDH

,Country,2021,2020,2019
181,Afghanistan,0.473,0.483,0.488
73,Albania,0.795,0.794,0.810
92,Algeria,0.740,0.736,0.748
34,Andorra,0.855,0.848,0.873
149,Angola,0.590,0.590,0.593
...,...,...,...,...
118,Venezuela,0.690,0.695,0.721
106,Vietnam,0.718,0.710,0.703
185,Yemen,0.425,0.460,0.461
152,Zambia,0.565,0.570,0.575


In [17]:
df_transformado = pd.melt(IDH, id_vars=['Country'], var_name='Year', value_name='IDH')
df_transformado = df_transformado.sort_values(by=['Country', 'Year']).reset_index(drop=True)
df_transformado

,Country,Year,IDH
0,Afghanistan,2019,0.488
1,Afghanistan,2020,0.483
2,Afghanistan,2021,0.473
3,Albania,2019,0.810
4,Albania,2020,0.794
...,...,...,...
571,Zambia,2020,0.570
572,Zambia,2021,0.565
573,Zimbabwe,2019,0.525
574,Zimbabwe,2020,0.524


In [18]:
df_transformado.to_csv("IDH.csv", index=False)